# Dieselgate Routing

From the year 2019 on, Berlin will impose the diesel ban. The following streets will be affected: Leipziger Straße, Reinhardstraße, Friedrichstraße, Brückenstraße, Kapweg, Alt-Moabit, Stromstraße und Leonorenstraße. 

This notebook gives a short overview how the routing may change for diesel driver when certain streets must be avoided.

In [ ]:


widget_list.observe(street['properties']['county'], names='value') # Hidden method is called by the listener        
display(widget_dd)


start_list = []
for poi in poi_start['features']:
    start_list = widgets.Select(
        options=[poi['properties']['county']],
        #value=,
        # rows=10,
        description='OS:',
        disabled=False)
display(start_list)

for poi in poi_dict['start']['features']:
    if poi['properties']['county'] == 'Berlin ':
        pprint(poi)
        #folium.GeoJson(street['geometry']).add_to(map_berlin)
#map_berlin

In [4]:
# Needed packages
from openrouteservice import client
import folium
from shapely.geometry import LineString, Polygon, mapping

def style_function(color): # To style data
    return lambda feature: dict(color=color)

### Regular Route

So far: The shortest route for a car from A to B.

In [2]:
# insert your ORS api key
api_key = '5b3ce3597851110001cf62486c1d5544489d41dc8c0f542b6111d7f9' #'Your-Key' 
clnt = client.Client(key=api_key)

map_berlin = folium.Map(tiles='https://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', attr='Map data (c) OpenStreetMap, Tiles (c) GIScience Heidelberg', location=([52.520921, 13.406582]), zoom_start=12.5) # Create map
coordinates = [[13.388643, 52.499242], [13.39551, 52.538831]]
direction_params = {'coordinates': coordinates,
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}

regular_request = clnt.directions(**direction_params) # Direction request
folium.GeoJson(regular_request, name='Regular Route').add_to(map_berlin)

folium.Marker(list(reversed(coordinates[0])), popup='Start').add_to(map_berlin)
folium.Marker(list(reversed(coordinates[1])), popup='Destination').add_to(map_berlin)
map_berlin

### Dieselgate Routing

Coming soon: The shortest route for a diesel driver, which must avoid the in orange color areas. Apparently the driving way extents and the polluting car must drive even a longer route.

In [5]:
# Start and destination coordinates of affectes streets
avoid_streets = [[[13.390478, 52.506805], [13.387141, 52.52696]], [[13.377807, 52.509587], [13.401142, 52.511291]], [[13.416549, 52.511141], [13.417686, 52.513531]], [[13.327618, 52.524322], [13.367872, 52.522325]], [[13.342155, 52.523474], [13.343239, 52.531555]]]

# Affected streets
buffer = []
for street in avoid_streets:
    avoid_params = {'coordinates': street,
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}
    avoid_request = clnt.directions(**avoid_params)
    coords = avoid_request['features'][0]['geometry']['coordinates']
    folium.PolyLine([(y,x) for x,y in coords], 
                    color=('#ffb31a'), 
                    popup='Affected Streets').add_to(map_berlin)
    route_buffer = LineString(coords).buffer(0.0001) # Create geometry buffer
    simp_geom = route_buffer.simplify(0.0001) # Simplify geometry for better handling
    buffer.append(mapping(Polygon(simp_geom)))
    
# New routing with avoided streets
for street in buffer[0:2]:
    diesel_request = {'coordinates': coordinates, 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                    'instructions': False,
                     'options': {'avoid_polygons': street}} 
route_directions = clnt.directions(**diesel_request)

folium.GeoJson(route_directions,
              style_function=style_function('red'), 
                name='New Route').add_to(map_berlin)

map_berlin.add_child(folium.map.LayerControl())